### Music Genre Classification using KNN

#### Problem Statement:- Given multiple audio files, and the task is to categorize each audio file in a certain category like audio belongs to Disco, hip-hop, etc.

##### Import libraries

In [2]:
!pip install python_speech_features
!pip install scipy

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5893 sha256=16eb922875c74a99ad2ac6736883332bf8b9bc4161d46048c7ad15e111ef02ba
  Stored in directory: c:\users\sneh\appdata\local\pip\cache\wheels\5b\60\87\28af2605138deac93d162904df42b6fdda1dab9b8757c62aa3
Successfully built python-speech-features


In [3]:
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tempfile import TemporaryFile
import os
import math
import pickle
import random
import operator

##### Calculate distance to find neighbours

In [4]:
def getNeighbors(trainingset, instance, k):
    distances = []
    for x in range(len(trainingset)):
        dist = distance(trainingset[x], instance, k) + distance(instance,trainingset[x],k)
        distances.append((trainingset[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

##### Find class having maximum neighbours

In [5]:
def nearestclass(neighbors):
    classVote = {}
    
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1
            
    sorter = sorted(classVote.items(), key=operator.itemgetter(1), reverse=True)
    return sorter[0][0]

##### Model evalution function

In [6]:
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == prediction[x]:
            correct += 1
    return 1.0 * correct / len(testSet)

##### Feature extraction using MFCC

In [7]:
directory = 'C:/Users/Sneh/Downloads/archive/Data/genres_original'
f = open("mydataset.dat", "wb")
i = 0
for folder in os.listdir(directory):
    #print(folder)
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+"/"+folder):
        #print(file)
        try:
            (rate, sig) = wav.read(directory+"/"+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print("Got an exception: ", e, 'in folder: ', folder, ' filename: ', file)
f.close()

Got an exception:  File format b'\xcb\x15\x1e\x16' not understood. Only 'RIFF' and 'RIFX' supported. in folder:  jazz  filename:  jazz.00054.wav


##### Split the dataset

In [12]:
dataset = []

def loadDataset(filename, split, trset, teset):
    with open('mydataset.dat','rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trset.append(dataset[x])
        else:
            teset.append(dataset[x])

trainingSet = []
testSet = []
loadDataset('my.dat', 0.68, trainingSet, testSet)


##### Make prediction

In [14]:
#Define distance

def distance(instance1, instance2, k):
    distance = 0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

In [15]:
#Prediction

length = len(testSet)
predictions = []
for x in range(length):
    predictions.append(nearestclass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

0.7468354430379747


##### Testing the model

In [17]:
from collections import defaultdict
results = defaultdict(int)

directory = 'C:/Users/Sneh/Downloads/archive/Data/genres_original'

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [20]:
pred = nearestclass(getNeighbors(dataset, feature, 5))
print(results[pred])

rock


##### Model sucessfully predicted the genre of the music.